In [2]:
import torch

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token
from transformers.generation.streamers import TextIteratorStreamer

from PIL import Image

import requests
from io import BytesIO

from cog import BasePredictor, Input, Path, ConcatenateIterator
import time
import subprocess
from threading import Thread

from llava.model.llava_arch import LlavaMetaModel, LlavaMetaForCausalLM
import os

from llava.model.language_model.llava_mistral import LlavaMistralConfig,LlavaMistralModel, LlavaMistralForCausalLM

In [3]:
pip install cog

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dataclasses import dataclass, field
import os
import copy
from dataclasses import dataclass, field
import json
import logging
import pathlib
from typing import Dict, Optional, Sequence, List
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
import torch.nn as nn
processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
model = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-large-patch14").half().cuda()
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("checkpoints/llava--finetune/checkpoint-29571")
image="dog1.jpeg"
image = Image.open("dog1.jpeg")
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image 

model1=LlavaMistralForCausalLM.from_pretrained("checkpoints/llava--finetune/checkpoint-29571").half()

@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="checkpoints/llava--finetune/checkpoint-29571")
    version: Optional[str] = field(default="v0")
    freeze_backbone: bool = field(default=False)
    tune_mm_mlp_adapter: bool = field(default=False)
    vision_tower: Optional[str] = field(default="openai/clip-vit-large-patch14")
    mm_vision_select_layer: Optional[int] = field(default=-1)   # default to the last layer
    pretrain_mm_mlp_adapter: Optional[str] = field(default=None)
    mm_projector_type: Optional[str] = field(default='linear')
    mm_use_im_start_end: bool = field(default=False)
    mm_use_im_patch_token: bool = field(default=True)
    mm_patch_merge_type: Optional[str] = field(default='flat')
    mm_vision_select_feature: Optional[str] = field(default="patch")
model_args=ModelArguments()
image_features = processor.image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
#image_tensor=model.get_image_features(pixel_values=image_features).half().cuda()
model1.get_model().initialize_vision_modules(
            model_args=model_args
        )
model1.to('cuda')
#state_dict=torch.load("checkpoints/llava--pretrain/checkpoint-37211/mm_projector.bin")
#new_state_dict = {k.replace("model.mm_projector.", ""): v for k, v in state_dict.items()}
#layer=nn.Linear(768,1536,bias=True).to('cuda').half()
#layer.load_state_dict(new_state_dict)
conv_mode='qwen_2'
conv = conv_templates[conv_mode].copy()
#image_tensor=layer(image_tensor.to('cuda')).half()
prompt="what is in this image"
inp = DEFAULT_IMAGE_TOKEN + '\n' + prompt
conv.append_message(conv.roles[0], inp)

conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
    
input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
print(input_ids)
print(image_features)
stop_str=conv.sep2
keywords = [stop_str]
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, timeout=1)
def stream_generation(input_ids, image_tensor, stop_str, temperature, top_p, max_tokens, streamer):
    from threading import Thread
    # Start the generation thread inside inference mode
    with torch.inference_mode():
        thread = Thread(
            target=model1.generate,  # or model.generate if not inside a class
            kwargs=dict(
                inputs=input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                max_new_tokens=max_tokens,
                streamer=streamer,
                use_cache=True
            )
        )
        thread.start()
        prepend_space = False
        for new_text in streamer:
            if new_text == " ":
                prepend_space = True
                continue
            if new_text.endswith(stop_str):
                new_text = new_text[:-len(stop_str)].strip()
                prepend_space = False
            elif prepend_space:
                new_text = " " + new_text
                prepend_space = False
            if len(new_text):
                yield new_text
        if prepend_space:
            yield " "
        thread.join()

# Now, you can iterate over the generator:
#for chunk in stream_generation(input_ids, image_features, stop_str, .7, 1, 100, streamer):
    #print(chunk)
tensor=model1.generate(inputs=input_ids,
                images=image_features,
                max_new_tokens=100)
                #do_sample=True,
                #temperature=.2,
                #top_p=1,
                
                #use_cache=False)
token_ids = tensor.cpu().tolist()[0]  # Squeeze out the batch dimension if necessary
decoded_text = tokenizer.decode(token_ids, skip_special_tokens=True)
print(decoded_text)

Some weights of the model checkpoint at checkpoints/llava--finetune/checkpoint-29571 were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.embeddings.class_embedding', 'model.vision_tower.vision_tower.vision_model.embeddings.patch_embedding.weight', 'model.vision_tower.vision_tower.vision_model.embeddings.position_embedding.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.e

tensor([[151646,     32,   6236,   1948,    264,  22208,   1196,    323,    458,
          20443,  11229,  17847,     13,    576,  17847,   6696,  10950,     11,
          11682,     11,    323,  47787,  11253,    311,    279,   1196,    594,
           4755,     13, 151646,   6448,     25,    220,   -200,    198,  12555,
            374,    304,    419,   2168, 151646,   4939,   3846,   2821,     25]],
       device='cuda:0')
tensor([[[[1.4775, 1.4189, 1.3613,  ..., 1.6387, 1.6240, 1.6240],
          [1.4775, 1.4189, 1.3613,  ..., 1.5947, 1.5654, 1.5654],
          [1.4775, 1.4189, 1.3613,  ..., 1.5801, 1.5801, 1.5801],
          ...,
          [1.1416, 1.1133, 1.0254,  ..., 1.2588, 1.2148, 1.1709],
          [1.2148, 1.2734, 1.2295,  ..., 1.1709, 1.2148, 1.0986],
          [1.2148, 1.2295, 1.3174,  ..., 1.1709, 1.1270, 1.2441]],

         [[1.6250, 1.5947, 1.5498,  ..., 1.7148, 1.6992, 1.6992],
          [1.6250, 1.5947, 1.5498,  ..., 1.6699, 1.6396, 1.6396],
          [1.6250, 1.594

In [2]:
cd LLaVA

/home/user/LLaVA


# 